
# 1. Imports

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, DecimalType


# 2. Carregando sparkDFs


## 2.1 Orders

In [0]:
# Local do arquivo Delta
path_to_delta_table = "dbfs:/FileStore/Datum/KaggleOlistData/silver/delta/orders"

# Ler o arquivo Delta como um DataFrame
df_orders = spark.read.format("delta").load(path_to_delta_table)

In [0]:
display(df_orders.take(10))

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_diff
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02T10:56:33Z,2017-10-02T11:07:15Z,2017-10-04T19:55:00Z,2017-10-10T21:25:13Z,2017-10-18T00:00:00Z,-8
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24T20:41:37Z,2018-07-26T03:24:27Z,2018-07-26T14:31:00Z,2018-08-07T15:27:45Z,2018-08-13T00:00:00Z,-6
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08T08:38:49Z,2018-08-08T08:55:23Z,2018-08-08T13:50:00Z,2018-08-17T18:06:29Z,2018-09-04T00:00:00Z,-18
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18T19:28:06Z,2017-11-18T19:45:59Z,2017-11-22T13:39:59Z,2017-12-02T00:28:42Z,2017-12-15T00:00:00Z,-13
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13T21:18:39Z,2018-02-13T22:20:29Z,2018-02-14T19:46:34Z,2018-02-16T18:17:02Z,2018-02-26T00:00:00Z,-10
a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09T21:57:05Z,2017-07-09T22:10:13Z,2017-07-11T14:58:04Z,2017-07-26T10:57:55Z,2017-08-01T00:00:00Z,-6
136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11T12:22:08Z,2017-04-13T13:25:17Z,null,null,2017-05-09T00:00:00Z,null
6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16T13:10:30Z,2017-05-16T13:22:11Z,2017-05-22T10:07:46Z,2017-05-26T12:55:51Z,2017-06-07T00:00:00Z,-12
76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered,2017-01-23T18:29:09Z,2017-01-25T02:50:47Z,2017-01-26T14:16:31Z,2017-02-02T14:08:10Z,2017-03-06T00:00:00Z,-32
e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered,2017-07-29T11:55:02Z,2017-07-29T12:05:32Z,2017-08-10T19:45:24Z,2017-08-16T17:14:30Z,2017-08-23T00:00:00Z,-7


In [0]:
del path_to_delta_table


# 3. Transformação de Dados


## 3.1 Orders

In [0]:
df_orders = df_orders.filter(df_orders['delivery_time_diff'].isNotNull())

df_orders = df_orders.filter(df_orders['order_status'] != "canceled")

In [0]:
display(df_orders.take(10))

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_diff
6ec1bea8cbcef0a1b81bc9b7fbd37ccb,e6b5e20566e5c72cbaab04f91dec9c85,delivered,2018-08-07T05:42:46Z,2018-08-07T05:50:08Z,2018-08-13T09:24:00Z,2018-08-27T15:28:22Z,2018-09-18T00:00:00Z,-22
441972a5bbd51a10459a487402076942,b79fa9dfed0c3d624b70fbd0ca2469de,delivered,2018-08-23T22:37:44Z,2018-08-24T00:35:13Z,2018-08-24T13:19:00Z,2018-08-29T18:03:26Z,2018-09-04T00:00:00Z,-6
fbecebecbe32df9dc3411b1e35b5484c,7f9f88f14a8f0dc733a5e371af98abac,delivered,2017-03-20T18:22:46Z,2017-03-20T18:22:46Z,2017-03-21T13:35:58Z,2017-03-27T12:30:15Z,2017-04-12T00:00:00Z,-16
9e22b00fd1b9f036577e8c517ea0a5d4,5a7346263920ecab0f88da703bbaf3b1,delivered,2017-12-01T13:45:30Z,2017-12-03T13:52:24Z,2017-12-04T21:22:58Z,2018-01-03T21:36:42Z,2017-12-27T00:00:00Z,7
07d9d11b0175952638ddd53d7855f8a7,a1c779cddccf5ffe60e543d3ce0dc477,delivered,2017-11-22T16:40:23Z,2017-11-22T16:48:33Z,2017-11-28T19:48:46Z,2017-11-30T13:49:03Z,2017-12-04T00:00:00Z,-4
1c38314597f39a117548efb308041b76,39a23021e4ee2efd04ea8decaff7b328,delivered,2017-12-11T15:43:06Z,2017-12-11T15:51:22Z,2017-12-18T18:35:20Z,2017-12-28T17:43:32Z,2018-01-08T00:00:00Z,-11
06caf4b0e55d9bc015a91fa77432a1cb,e42b1319fefce9bf37acf91a0ac8c1a9,delivered,2017-03-11T11:42:16Z,2017-03-11T11:42:16Z,2017-03-13T14:54:54Z,2017-03-21T10:58:09Z,2017-04-06T00:00:00Z,-16
1330176f6500125ff6e8e10922fb714c,89479f50ebac0687b3d8d0ac9f1193b1,delivered,2017-10-18T23:24:58Z,2017-10-19T00:05:28Z,2017-10-20T17:52:05Z,2017-10-21T17:34:44Z,2017-10-30T00:00:00Z,-9
6616fa4c89b8bf2a7e17271cdc542fca,19245d7a8dfb1639e01f96c455fb05b1,delivered,2018-08-17T20:47:06Z,2018-08-18T03:09:38Z,2018-08-23T07:11:00Z,2018-08-28T16:09:37Z,2018-09-25T00:00:00Z,-28
06f544e5c7afe6215b9e2a26b4f08442,26d283dae44fbc9eee874cecca589853,delivered,2018-04-12T10:23:19Z,2018-04-12T10:35:13Z,2018-04-17T19:41:03Z,2018-04-24T18:41:15Z,2018-05-08T00:00:00Z,-14


In [0]:
# Definindo a função para categorizar 'delivery_time_diff'
def categorize_delivery_time_diff(diff):
    if diff > 10:
        return '11, 12, ..., 188'
    elif diff >= 8:
        return '8, 9 ou 10'
    elif diff >= 6:
        return '6 ou 7'
    elif diff >= 4:
        return '4 ou 5'
    elif diff == 3:
        return '3'
    elif diff == 2:
        return '2'
    elif diff == 1:
        return '1'
    elif diff == 0:
        return '0'
    elif diff == -1:
        return '-1'
    elif diff == -2:
        return '-2'
    elif diff == -3:
        return '-3'
    elif diff == -4 or diff == -5:
        return '-4 ou -5'
    elif diff == -6 or diff == -7:
        return '-6 ou -7'
    elif diff >= -10 and diff <= -8:
        return '-8, -9 ou -10'
    else:
        return '-11, -12, ..., -147'

In [0]:
# Registrando a função como UDF
categorize_udf = udf(categorize_delivery_time_diff, StringType())

In [0]:
# Adicionando a nova coluna 'category'
df_orders = df_orders.withColumn('category', categorize_udf('delivery_time_diff'))

# Calculando a contagem total
total_orders = df_orders.count()

# Agrupando por categoria e contando
df_category_count = df_orders.groupBy('category').count()

# Calculando a porcentagem e convertendo para DecimalType(5,2)
df_percentage = df_category_count.withColumn('percentage', (df_category_count['count'] / total_orders * 100).cast(DecimalType(5, 2)))

# Ordenando o resultado final de forma descendente pela coluna 'percentage'
df_delivery_date_analyses = df_percentage.selectExpr('category as delivery_time_diff', 'percentage').orderBy('percentage', ascending=False)

In [0]:
display(df_delivery_date_analyses.take(10))

delivery_time_diff,percentage
"-11, -12, ..., -147",59.300000000000000000
"-8, -9 ou -10",14.620000000000000000
-6 ou -7,8.770000000000000000
-4 ou -5,4.290000000000000000
"11, 12, ..., 188",2.170000000000000000
-3,1.790000000000000000
-2,1.610000000000000000
-1,1.520000000000000000
0,1.340000000000000000
6 ou 7,0.940000000000000000


In [0]:
del df_orders, total_orders, df_category_count, df_percentage


# 4. Exportando para Delta Lake

> Teste Prático -> "Grave os mesmos dados em formato Delta Lake para aproveitar as funcionalidades de versionamento e transações ACID."

In [0]:
df_delivery_date_analyses.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(
    "dbfs:/FileStore/Datum/KaggleOlistData/gold/delta/delivery_date_analyses"
)

In [0]:
del df_delivery_date_analyses


# 5. Criando Delta Table

In [0]:
%sql

create database if not exists olist;

In [0]:
%sql

use olist;

In [0]:
%sql

drop table if exists delivery_date_analyses;

In [0]:
%sql

CREATE TABLE delivery_date_analyses
USING DELTA
LOCATION "dbfs:/FileStore/Datum/KaggleOlistData/gold/delta/delivery_date_analyses"